## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2019-09-22 23:14:11--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2019-09-22 23:14:11--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4aeba5a8cb115fd5d70006d5ea.dl.dropboxusercontent.com/cd/0/get/ApDakkxEs6qHQZ8dwZXxHBNOTs0yPq10zTMjXr_3jkoxPaHn6O4SZnI3CVFXWBuml-xIbugIAxY124-I6zZRsGExGuO8W70Yh2uGAnDQwaMQ3Q/file?dl=1# [following]
--2019-09-22 23:14:12--  https://uc4aeba5a8cb115fd5d70006d5ea.dl.dropboxusercontent.com/cd/0/get/ApDakkxEs6qHQZ8dwZXxHBNOTs0yPq10zTMjXr_3jkoxPaHn6O4SZnI3CVFXWBuml-xIbugIAxY124-I6zZRsGExGuO8W70Yh2uGAnDQwaMQ3Q/file?dl=1
Resolving uc4aeba5

In [2]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(u.lower()) for u in data]

In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-1.7912642e+00, -4.2751607e-02,  2.3930392e+00, -1.9206357e+00,
       -1.8191214e+00, -1.1875527e+00, -3.5526925e-01,  3.8049974e+00,
       -1.3532555e+00,  9.9727648e-01,  1.2638477e+00, -1.8640429e+00,
        1.0883015e+00,  3.1882193e+00, -1.5662316e+00, -3.1100471e+00,
       -8.6036241e-01, -3.0280161e+00, -3.4063780e-01, -1.6920475e-03,
        3.7185659e+00, -3.9733558e+00,  4.4919080e-01, -8.9925569e-01,
       -1.9438699e+00, -2.5892529e+00, -2.2493629e-01, -4.7529086e-02,
        1.0399430e+00,  1.5945280e-01,  7.9234082e-01, -4.2615638e+00],
      dtype=float32)

In [9]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.946851372718811),
 ('cheese', 0.9306034445762634),
 ('butter', 0.9183666706085205),
 ('sauce', 0.9163393378257751),
 ('fruit', 0.9121607542037964),
 ('cream', 0.9037689566612244),
 ('honey', 0.9010499119758606),
 ('beans', 0.8993000388145447),
 ('vodka', 0.8985033631324768),
 ('chocolate', 0.8946813344955444)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [10]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[========================================----------] 81.2% 314.5/387.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 387.1/387.1MB downloaded


In [11]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843312203884125),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [12]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [31]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [32]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [68]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit(word_vectors).transform(word_vectors)
word_vectors_pca = (word_vectors_pca - word_vectors_pca.mean(0)) / word_vectors_pca.std(0)

In [69]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [70]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [71]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1489', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [72]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca


word_tsne = TSNE(n_components=2, verbose=100).fit_transform(word_vectors)
word_tsne = (word_tsne - word_tsne.mean(0)) / word_tsne.std(0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.002s...
[t-SNE] Computed neighbors for 1000 samples in 0.181s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.066s
[t-SNE] Iteration 50: error = 68.3134689, gradient norm = 0.3172579 (50 iterations in 4.935s)
[t-SNE] Iteration 100: error = 69.3443375, gradient norm = 0.2831144 (50 iterations in 5.208s)
[t-SNE] Iteration 150: error = 68.5552368, gradient norm = 0.3080564 (50 iterations in 5.308s)
[t-SNE] Iteration 200: error = 68.7150497, gradient norm = 0.2875424 (50 iterations in 5.493s)
[t-SNE] Iteration 250: error = 69.3353195, gradient norm = 0.2881215 (50 iterations in 5.214s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.335320
[t-SNE] Iteration 300: error = 1.2141252, gradient norm = 0.0034120 (50 iterations in 3.966s)
[t-SNE] Iteration 350: error = 1.1012290, gradient norm = 0.00053

In [73]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1632', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [90]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    cnt = 0
    
    for token in tokenizer.tokenize(phrase.lower()):
        if token in model:
            vector += model.get_vector(token)
            cnt += 1
    if cnt:
        vector /= cnt
    
    return vector
        
    

In [91]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")
assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [92]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [93]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [94]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.015s...
[t-SNE] Computed neighbors for 1001 samples in 0.200s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 0.477874
[t-SNE] Computed conditional probabilities in 0.047s
[t-SNE] Iteration 50: error = 79.3867798, gradient norm = 0.3341210 (50 iterations in 5.327s)
[t-SNE] Iteration 100: error = 80.3990250, gradient norm = 0.3064474 (50 iterations in 5.812s)
[t-SNE] Iteration 150: error = 81.4542465, gradient norm = 0.2885543 (50 iterations in 5.589s)
[t-SNE] Iteration 200: error = 82.5826492, gradient norm = 0.2851577 (50 iterations in 5.727s)
[t-SNE] Iteration 250: error = 83.0082855, gradient norm = 0.2946007 (50 iterations in 6.321s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.008286
[t-SNE] Iteration 300: error = 2.1810768, gradient norm = 0.0025562 (50 iterations in 4.496s)

In [95]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1945', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [96]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [161]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_emb = get_phrase_embedding(query)
    similarities = []
    for word_emb in data_vectors:
        if not np.isfinite(word_emb).all():
            continue
        word_len = (word_emb * word_emb).sum()
        similarity = (query_emb * word_emb).sum()
        if abs(word_len) > 1e-5:
            similarity /= word_len
        similarities.append(similarity)
    similarities = np.array(similarities)
    partition_indexes = np.argpartition(similarities, kth=-k)[-k:]
    print(similarities.argmax(), partition_indexes)
    print(similarities.max())
    k_first_words = [data[x] for x in partition_indexes]
    k_first_similarities = similarities[partition_indexes]
    print(k_first_similarities)
    print(k_first_words)
    return [k_first_words[x] for x in np.argsort(k_first_similarities)][::-1]

In [162]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

225497 [455619 375873 500667 445473 498893 286156 246760 225497 386041 531261]
1.6224589
[1.46555   1.46686   1.4702437 1.6019021 1.4714763 1.4904205 1.4886556
 1.6224589 1.4915996 1.4895011]
['Why did Kangana Ranaut not get a single award for "Tanu weds Manu returns" whereas Deepika & Sonam got awards for "Piku" and "Prem Ratan Dhan Payo"?\n', "How many international science olympians (IMO, IPhO, IChO, IOI, IBO) join Carnegie Mellon's freshman class each year?\n", 'Any one any Idea about "Micron Robotics " Human Resource Department Micron Robotics www.micronrobotics.com B-5, Sector-64,Noida.?\n', 'What do the below terms mean: GFE PSE BBBJ Russian Greek MSOG Daty CIM OWO BBBJTC BBFS?\n', 'How did yogesh kumar (ssc cgl 2015 topper) score 190.5 marks in tier 1?\n', 'How does one determine the standard error for regression coefficients yielded by sparse regression methods (e.g. Ridge Regression, LASSO, Elastic Net, etc.)?\n', "Nayan Bansal JEE Advanced rank 13060 rank got admission in II

AssertionError: 

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.